# Introduction

**Main Topic**

This notebook is for **Detecting Helmet using [MMdetection](https://github.com/open-mmlab/mmdetection) without internet** using **images** and **image_labels.csv**

Before you start this Kernel you need to make COCO format train, val.json, you can check [Create COCO Anntations](https://www.kaggle.com/jinssaa/create-coco-anntations) nb or refer [mydataset](https://www.kaggle.com/jinssaa/nflweightutils).


**References**

**Chris Deotte Grand master** : [How to Install Without Internet](https://www.kaggle.com/c/severstal-steel-defect-detection/discussion/113195)

[**MMdetection Official Documents**](https://mmdetection.readthedocs.io/)


# Install MMdetection from scratch

**Version info.**

- MMdetection 2.6.0
- mmcv-full 1.2.0, torch 1.6, cu102

Because this Competetion is [Notebook Competetion](https://www.kaggle.com/docs/competitions#notebooks-only-FAQ), we need to inference .mp4 video without interent. 

So I made `*.whl` files to install MMdetection, mmcv-full without internet. you can use this files from [mmdetection-v2.6.0 dataset](https://www.kaggle.com/jinssaa/mmdetectionv260).

- note I think we don't need to `train` without internet so use local env if you want. this step for `inference` using weight.

In [ ]:
! pip install ../input/mmdetectionv260/addict-2.4.0-py3-none-any.whl

In [ ]:
! pip install ../input/mmdetectionv260/mmcv_full-latesttorch1.6.0cu102-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
! pip install ../input/mmdetectionv260/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl

In [ ]:
! pip install ../input/mmdetectionv260/mmdet-2.6.0-py3-none-any.whl

# Set up environment

In [ ]:
import copy
import os.path as osp

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.datasets import build_dataset, CocoDataset
from mmdet.models import build_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import train_detector, set_random_seed, init_detector, inference_detector

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import nflimpact
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.core.display import display, HTML

# Set Dataset

This class is from [MMdetection COCO.py](https://github.com/open-mmlab/mmdetection/blob/master/mmdet/datasets/coco.py)


I just set `CLASSES` only `Helmet`, So you can customize if you want.


In [ ]:
@DATASETS.register_module()
class ImpactDataset(CocoDataset):
    CLASSES = ('Helmet')

# Get Config

I shared [NFL-weight-utils Dataset](https://www.kaggle.com/jinssaa/nflweightutils). there are files below.

- configs.py : Default faster R-CNN configs. you can customize, please check [Official MMdetection configs](https://github.com/open-mmlab/mmdetection/tree/master/configs)
- train.json : train.json made by [Create COCO Anntations](https://www.kaggle.com/jinssaa/create-coco-anntations)
- valid.json : valid.json made by [Create COCO Anntations](https://www.kaggle.com/jinssaa/create-coco-anntations)
- resnet50-19c8e357.pth : Official pytorch resnet50 weight
- latest.pth : pretrained faster R-CNN model trained by below configs, 10 epochs

In [ ]:
config_file = '../input/nflweightutils/configs.py'
cfg = Config.fromfile(config_file)

You need to modify some configs below.

In [ ]:
cfg.total_epochs = 1
cfg.work_dir = './'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data_root = '../input/nfl-impact-detection/'

cfg.data.train.ann_file='../input/nflweightutils/train.json'
cfg.data.train.img_prefix= cfg.data_root + 'images/'

cfg.data.val.ann_file='../input/nflweightutils/valid.json',
cfg.data.val.img_prefix= cfg.data_root + 'images/'

cfg.data.test.ann_file='../input/nflweightutils/valid.json', ## I just set validation data for test because it's not our main purpose
cfg.data.test.img_prefix= cfg.data_root + 'images/'
cfg.model.pretrained = '../input/nflweightutils/resnet50-19c8e357.pth'

Let's check configs

In [ ]:
print(f'Config:\n{cfg.pretty_text}')

# Train model

When we submit our result, we need to install MMdetection without internet. so install MMdetection locally.

just build datasets and train model. I just train 1 epoch in this cell because it's not our main purpose. You can use MMdetection in local envs

In [ ]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Test video

Let's test video and visualize

In [ ]:
fig = plt.figure()

# Specify the path to model config and checkpoint file
checkpoint_file = '../input/nflweightutils/latest.pth' # 10 epochs

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a video and show the results
video = mmcv.VideoReader('../input/nfl-impact-detection/test/57906_000718_Endzone.mp4')

ims = []

for frame in video:
    result = inference_detector(model, frame)
    single_img = model.show_result(frame, result, wait_time=1)
    im = plt.imshow(single_img, animated=True)
    ims.append([im])
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

# Display video

Sorry for the low resolution. please let's me know if there a way to improve video rendering resolution !

In [ ]:
display(HTML(ani.to_html5_video()))

# Next Step

I think this Competition is quite a hard competition. it's hard to find way to predict `impact`. So please share your idea :)

I'will try method below

1. Get `impact` frames from  `train` folder. I made [full frame train images](https://www.kaggle.com/jinssaa/nfl-frame)(14GB) from train *.mp4. take only `impact` images and build `impact` detection model.

2. train `impact` detection model, and configure using rule-based way(ex. get intersection pred vs pred)

3. **any ideas?**

Thank you! Happy Kaggle !
